In [1]:
library(ggplot2)
library(sparklyr)
library(magrittr)


Attaching package: ‘sparklyr’


The following object is masked from ‘package:stats’:

    filter




In [3]:
sc <- spark_connect(master = "local")

In [4]:
#copy data from an R data.frame or tibble to Spark.
diamonds_in_spark <- copy_to(sc, diamonds)

In [6]:
diamonds_in_spark

# Source: spark<diamonds> [?? x 10]
   carat cut       color clarity depth table price     x     y     z
   <dbl> <chr>     <chr> <chr>   <dbl> <dbl> <int> <dbl> <dbl> <dbl>
 1  0.23 Ideal     E     SI2      61.5    55   326  3.95  3.98  2.43
 2  0.21 Premium   E     SI1      59.8    61   326  3.89  3.84  2.31
 3  0.23 Good      E     VS1      56.9    65   327  4.05  4.07  2.31
 4  0.29 Premium   I     VS2      62.4    58   334  4.2   4.23  2.63
 5  0.31 Good      J     SI2      63.3    58   335  4.34  4.35  2.75
 6  0.24 Very Good J     VVS2     62.8    57   336  3.94  3.96  2.48
 7  0.24 Very Good I     VVS1     62.3    57   336  3.95  3.98  2.47
 8  0.26 Very Good H     SI1      61.9    55   337  4.07  4.11  2.53
 9  0.22 Fair      E     VS2      65.1    61   337  3.87  3.78  2.49
10  0.23 Very Good H     VS1      59.4    61   338  4     4.05  2.39
# … with more rows

In [ ]:
# bring back to R
diamonds_back_in_R <- collect(diamonds_in_spark)

In [ ]:
#read and write
spark_write_csv(diamonds_in_spark, "sparkdiamonds.csv", overwrite=TRUE)
diamonds_in_spark <- spark_read_csv(sc, "sparkdiamonds.csv")

### data wrangling

In [7]:
#We can now use the standard dplyr toolchain and, for example, define

In [8]:
large_diamonds_in_spark <- diamonds_in_spark %>%
                                filter(carat>1.0) %>%
                                mutate(mass = carat*0.2) %>%
                                select(mass, cut, color, price)
large_diamonds_in_spark

# Source: spark<?> [?? x 4]
    mass cut       color price
   <dbl> <chr>     <chr> <int>
 1 0.234 Very Good J      2774
 2 0.202 Premium   F      2781
 3 0.202 Fair      E      2788
 4 0.202 Premium   H      2788
 5 0.21  Very Good J      2789
 6 0.21  Fair      J      2789
 7 0.202 Fair      E      2797
 8 0.208 Premium   G      2801
 9 0.24  Fair      F      2809
10 0.204 Premium   G      2815
# … with more rows

In [19]:
cut_groups <- copy_to(sc, data.frame(cut=c("Fair", "Good", "Very Good", "Premium", "Ideal"),
coarse_cut=c("Fair", "(Very) Good", "(Very) Good",
"Premium+", "Premium+")),
"cut_groups", overwrite = TRUE)
cut_groups

# Source: spark<cut_groups> [?? x 2]
  cut       coarse_cut 
  <chr>     <chr>      
1 Fair      Fair       
2 Good      (Very) Good
3 Very Good (Very) Good
4 Premium   Premium+   
5 Ideal     Premium+   

In [23]:
summary_in_spark <- large_diamonds_in_spark %>%
                inner_join(cut_groups, by="cut") %>%
                group_by(coarse_cut)
                # summarise(min(price/mass, na.rm = TRUE))

summary_in_spark

ERROR: Error in group_by(., coarse_cut): could not find function "group_by"
